### Тематическое моделирование 



Построим тематические модели для корпуса новостей Lenta.ru, с которым мы уже работали ранее.

In [1]:
#!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [2]:
#!pip install corus

In [3]:
from corus import load_lenta

path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)
next(records)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество',
    date=None
)

In [4]:
data = [(record.title, record.topic, record.text, record.tags) for record in records]

In [5]:
import pandas as pd
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

<ipython-input-5-5b7176a9d163>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [6]:
data = pd.DataFrame(data, columns=['title','topic','text','tags'])

In [7]:
data.sample(3)

,title,topic,text,tags
85434,Thomson Reuters сообщил о лидерстве России по приросту золотого запаса,Экономика,"Банк России пятый год лидирует по приросту запасов золота среди центробанков мира. Об этом в пятницу, 31 марта, сообщает ТАСС со ссылкой на доклад Thomson Reuters «GFMS Gold Survey 2017». В прошлом году регулятор приобрел 201 тонну драгметалла. «При этом в октябре и ноябре покупки производились максимальными темпами — в среднем по 36 тонн в месяц в то время, как цены на золото упали», — отмечается в материалах агентства. В 2017 году Россия закупит примерно 200 тонн золота. На этот процесс практически не повлияют колебания цен на драгоценный металл или нефть, а также изменения валютного курса, полагают авторы исследования. При этом в будущем году мировые ЦБ приобретут примерно 250 тонн металла. В случае снятия антироссийских санкций Банк России может пересмотреть стратегию в плане покупки золота, отмечается в докладе. 23 марта регулятор сообщил, что международные резервы России выросли до максимального уровня с начала ноября 2016 года — 395,7 миллиарда долларов. По данным Банка России, объем монетарного золота в международных резервах вырос за 2016 год на 24 процента и составил 60,194 миллиарда долларов. При этом экспорт золота из страны в 2016 году снизился на 43 процента, до 22,4 тонны.",Госэкономика
67113,Потерявший сознание игрок «Аякса» получил необратимое повреждение мозга,Спорт,"Футболист голландского «Аякса» Абделхак Нури получил необратимые повреждения головного мозга. Об этом сообщается в аккаунте клуба в Twitter. «В это трудное время мы думаем и молимся о нем и людях, которых он любил», — говорится в сообщении. По информации AjaxDaily, большая часть мозга 20-летнего спортсмена не функционирует. Шансов на выздоровление нет. 8 июля Нури был доставлен в больницу после потери сознания в товарищеском матче против немецкого «Вердера». Врачи оказывали ему помощь прямо на поле. После чего футболист был доставлен в больницу на вертолете и введен в состояние искусственной комы. Сообщалось, что причиной потери сознания стала сердечная аритмия. В сезоне-2016/2017 Нури провел девять матчей за основную команду «Аякса», в которых не отметился забитыми мячами.",Футбол
321951,Рязанские единоросы поддержали предложение губернатора об отставке мэра,Россия,"Фракция ""Единой России"" в Рязанской городской думе поддержала предложение губернатора Олега Ковалева отправить в отставку мэра Федора Провоторова. Об этом сообщает ""Рязанский городской сайт"" со ссылкой на главу фракции Владимира Скрипченко. По его словам, 19 апреля состоялось заседание объединения депутатов, и они проголосовали за отставку Провоторова. Сколько человек присутствовало на заседании, не уточняется. Известно, что два депутата воздержались. На заседании правительства области 17 апреля губернатор Олег Ковалев раскритиковал Провоторова за лоббирование строительства автозаправки в интересах одного из депутатов. Кроме того, он напомнил о том, что Провоторов является фигурантом нескольких уголовных дел. ""Я долго занимал позицию невмешательства. Но сейчас вижу, что надо оказывать содействие правоохранительным органам для ускорения всех этих разбирательств"", - заявил Ковалев. В итоге губернатор предложил Провоторову уйти в отставку. По словам Провоторова, критика со стороны губернатора связана с отказом депутатов включить в программу приватизации несколько муниципальных унитарных предприятий (МУП). Ранее аналогичную версию называл рязанский оппозиционер Сергей Ежов. Он рассказал ""Коммерсанту"", что приватизацию лоббирует заместитель губернатора Александр Ревякин. ""Важно понимать, что это Ковалев сделал не из-за того, что внезапно ""прозрел"". Просто на кону пять рязанских МУПов, которые собираются приватизировать. Вот и началась войнушка"", - написал Сергей Ежов в своем блоге. Он рассказал, что на общественных слушаниях по поводу строительства автозаправки (в лоббировании которого Ковалев обвинял Провоторова) жители выступали против. По данным Ежова, на строитель

In [8]:
len(data)

739350

In [9]:
data.topic.value_counts().head(15)

Россия               160518
Мир                  136680
Экономика            79538 
Спорт                64421 
Культура             53803 
Бывший СССР          53402 
Наука и техника      53136 
Интернет и СМИ       44675 
Из жизни             27611 
Дом                  21734 
Силовые структуры    19596 
Ценности             7766  
Бизнес               7399  
Путешествия          6408  
69-я параллель       1268  
Name: topic, dtype: int64

Для ускорения вычислений оставим тексты из 5 тем:

In [10]:
topics = ['Экономика', 'Спорт', 'Культура', 'Путешествия', 
          'Наука и техника']


topics = ['69-я параллель', 'Мир', 'Интернет и СМИ', 'Бизнес', 
          'Дом']


data = data[data.topic.isin(topics)]
data.dropna(inplace=True)
len(data)

211756

### Предобработка


Все модели, с которыми мы будем работать далее, работают с предобработанными текстами, поэтому сперва проведем стандартную предобработку текста:

In [11]:
import re
import numpy as np
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
from multiprocessing import Pool
# from pymystem3 import Mystem

In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rost_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
#! pip install pymorphy2

In [14]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [15]:
words_regex = re.compile('\w+')

def find_words(text, regex = words_regex):
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]


stopwords_list = stopwords.words('russian')

# mystem = Mystem()
# def lemmatize(words, lemmer = mystem, stopwords = stopwords_list):
#     lemmas = lemmer.lemmatize(' '.join(words))
#     return [w for w in lemmas if not w in stopwords 
#             and w.isalpha()]

def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords 
            and w.isalpha()]

def preprocess(text):
    return (lemmatize(find_words(text)))

In [16]:
data.text.iloc[1]

'Хакерская группировка Anonymous опубликовала новые документы о деятельности британского аналитического центра Integrity Initiative. Из материалов следует, что центр получает финансирование от некого Института государственного управления под руководством Кристофера Доннелли. Хакеры выложили в открытый доступ его паспорт, резюме, а также сведения о его связях с Министерством обороны Великобритании. Согласно документам, в 2014 году Доннелли выдвинул ряд предложений британским властям в связи с ситуацией в Крыму. В частности, он планировал заминировать Севастопольскую бухту, окружить полуостров войсками, а также уничтожить оставшиеся в Крыму самолеты «в знак серьезности намерений». После публикации заметки часть документов, на которые члены группировки ссылаются в своем расследовании, была удалена. Также хакеры утверждают, что Доннелли был инициатором расследования «российского вмешательства» в референдум о независимости в Каталонии. Он пригласил членов испанского отделения Integrity Init

In [17]:
#print(preprocess(data.text.iloc[1]))

Ниже мы оставим только случайные 10,000 текстов из нашей коллекции, чтобы ускорить вычисления:

In [18]:
data = data.sample(10000)
data.topic.value_counts()

Мир               6404
Интернет и СМИ    2135
Дом               1034
Бизнес            362 
69-я параллель    65  
Name: topic, dtype: int64

In [19]:
preprocessed_text = list(tqdm(map(preprocess, data['text']), total=len(data)))

  0%|          | 0/10000 [00:00<?, ?it/s]

In [20]:
data['text'] = preprocessed_text
data.sample(3)

,title,topic,text,tags
49257,Атака «джихадмобиля» на пост иракской армии попала на видео,Мир,"[смертник, террористический, организация, исламский, государство, запретить, россия, джихадмобиль, атаковать, пост, иракский, армия, запад, провинция, анбар, видео, момент, взрыв, опубликовать, telegram, канал, directorate, заниматься, мониторинг, деятельность, террорист, видно, кадр, вооружённый, иракский, военный, сначала, паника, разбегаться, взрыв, спешить, место, происшествие, помочь, раненый, отмечать, канал, точный, данные, число, погибнуть, пострадать, иракский, командование, сообщить, сентябрь, сеть, появиться, видео, который, запечатлеть, смертник, заминировать, автомобиль, атаковать, колонна, иракский, армия, начинить, взрывчатка, машина, остановиться, вблизи, танк, abrams, взлететь, воздух]",Конфликты
639972,Американские самолеты бомбят кладбище в Неджефе,Мир,"[самолёт, ввс, сша, бомбить, кладбище, неджеф, данные, американский, разведка, скрываться, боевик, радикальный, мулла, муктада, аль, садра, сообщать, агентство, associated, press, минимум, бомба, сбросить, район, кладбище, неджеф, прибыть, делегация, иракский, религиозный, политический, деятель, уговорить, шиит, прекратить, восстание, посланник, багдад, собираться, потребовать, шиит, разоружиться, перевести, решение, спорный, вопрос, политический, плоскость, представитель, аль, садра, заявить, положительно, относиться, делегация, предложение, вторник, бой, центр, неджеф, усилиться, американский, войско, выдвинуть, танк, бронетехника, позиция, вокруг, старое, город, засесть, сотня, боец, армия, махдить, информация, associated, press, весь, время, бой, вторник, убить, мирный, житель, ранить, около, событие, неджеф, заставить, собраться, багдад, воскресение, национальный, конференция, прервать, свой, работа, выслать, аль, ...]",Все
724467,Билетным спекулянтам объявлена война в Сети,Интернет и СМИ,"[октябрь, состояться, презентация, проект, партер, единый, весь, россия, служба, бронирование, билет, всё, вид, развлекательный, мероприятие, московский, телефон, интернет, сайт, parter, работать, также, адрес, boxoffice, готовность, предоставлять, часть, свой, билет, партер, изъявить, московский, театр, среди, который, большой, театр, табаков, сатирикон, мастерская, фоменко, семь, кинотеатр, среди, который, ролан, кодак, киномир, художественный, ряд, спортивный, клуб, получить, забронированный, билет, уполномоченный, туристический, агентство, курьерский, доставка, стоимость, бронирование, рубль, доставка, курьер, обойтись, примерно, рубль, сказать, партер, первый, наш, страна, служба, род, например, свой, служба, бронирование, сайт, многий, театр, попытка, организация, столь, масштабный, единый, служба, единый, центр, предприниматься, впервые, характерный, основа, программный, обеспечение, проект, положить, система, сирена, ...]",Все


## Модель LDA


Mодель, которую мы рассмотрим, LDA - латентное размещение Дирихле. Воспользуемся реализацией из библиотеки sklearn.

In [21]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


stopwords_list = stopwords.words('russian')
n_features = 1000
n_components = 15
n_top_words = 10

new_corp = []

for sentence in preprocessed_text:
    sentence = ' '.join(sentence)
    new_corp.append(sentence)
    
corpus = new_corp

#tf_vectorizer = CountVectorizer(max_df=0.9, min_df=3, max_features=n_features, stop_words=stopwords_list, ngram_range=(1, 2))
#tf = tf_vectorizer.fit_transform(corpus)


tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=10, max_features=n_features, stop_words=stopwords_list, ngram_range=(1, 2))
tfidf = tfidf_vectorizer.fit_transform(corpus)



#lda = LatentDirichletAllocation(n_components=n_components, random_state=1).fit(tf)

lda = LatentDirichletAllocation(n_components=n_components, random_state=1).fit(tfidf)


In [22]:
len(lda.components_), len(lda.components_[0])

(15, 1000)

In [23]:
feature_names = tfidf_vectorizer.get_feature_names()
word_dict = {}
for topic_idx, topic in enumerate(lda.components_):
    top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
    top_features = [feature_names[i] for i in top_features_ind]
    word_dict[topic_idx] = top_features
    #break

pd.DataFrame(word_dict)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,россия,полиция,сша,человек,интернет,самолёт,суд,реклама,ирак,год,пользователь,увеличиться,израильский,министр,нью
1,украина,полицейский,обама,погибнуть,компания,борт,год,рекламный,иракский,процент,сеть,сумма,боевик,выборы,нью йорк
2,российский,человек,президент,взрыв,сайт,экипаж,дело,ролик,хусейн,компания,сервис,боевой,сирия,партия,йорк
3,страна,мужчина,трамп,произойти,медиа,авиакомпания,который,канал,военный,миллион,сайт,предприятие,израиль,страна,премия
4,иран,задержать,президент сша,пострадать,россия,аэропорт,обвинение,компания,саддам,рубль,google,малый,палестинский,правительство,мэр
5,президент,летний,буш,пожар,год,пассажир,это,the,американский,доллар,интернет,столица,военный,премьер,штат
6,это,который,барак,результат,директор,пилот,свой,телеканал,солдат,тысяча,facebook,коалиция,сирийский,президент,new
7,сша,город,барак обама,землетрясение,газета,судно,бывший,кампания,аль,метр,twitter,банк,группировка,премьер министр,церковь
8,заявить,женщина,американский,город,это,рейс,сша,фильм,войско,квадратный,компания,сектор,аль,парламент,вашингтон
9,путин,это,белый,сообщать,который,находиться,тюрьма,видео,военнослужащий,квадратный метр,который,заключение,террорист,год,сентябрь год


In [24]:
lda.perplexity(tfidf)

1866.1958758260532

### Результаты работы различных программ: ###
    
Библиотека __gensim__
Персплексия:  __4.771001008457515e-06__


Библиотека __BigARTM__

Персплексия:       artm.LDA __7.834888386479093__

Персплексия: __6.625234530618604__ (pLSA) vs. __6.675183672904855__ (mARTM)
    
    
    
Библиотека __sklearn__   

Персплексия: __7.531657346866097__    

Если отталкиваться от метрик (и по умолчанию считать что предобработка и параметры заданные моделям позволяют их сравнивать) то лучше всех сработала модель из библиотеки __gensim__.  

Если же смотреть глазами на первые результаты выдачи - то качество у всех моделей сопоставимо.